### Введение в нейронные сети

In [ ]:
# Это краткий конспект полезных выводов из курса по нейросетям - https://habr.com/ru/articles/414165/
# Также - полезная статья Яндекса со ссылками на источники - https://habr.com/ru/companies/yandex/articles/307260/

#### Общая ML теория

Задача ML = задача оптимизации весов модели на тренировочной выборке для лучшего предикта отложенной.  
- Пусть у нас есть объекты $u_1, u_2, ...$ (например, клиенты)
- Каждый объект $u_i$ может быть описан признаками в N-мерном пр-ве $x_{11}, x_{12}, ... x_{1N}$ 
- Мы решаем задачу классификации -> определяем какому из классов принадлежат объекты: $y_1, ... y_M$

Есть тренировочный размеченный датасет Train = $u_1, u_2...$     
Мы обучаем модель типа $y = Q(w_{1}, w_{2}, ... w_{K}, u_{1}, ... u_{N})$ с какими то весами  
А далее делаем предсказание на отложенной выборке predict(Q, u_test) -> y_predict  


___
<u>Линейный классификатор</u>  
y = (w, x) или в матричном виде: $W_{NM} * X_{N} = Y_{M}$  
Здесь - $X_{N}$ столбец признаков объекта, $Y_{M}$ - столбец вероятностей принадлежности к каждому классу  

___
<u>Метод макс правдоподобия</u>  
Макс. правдоподобие - общая метрика, которую надо оптимизировать для поиска корректных весов модели  

Likelihood = p(y=y_train_1 | u_train_1) * ... * p(y=y_train_D | u_train_D)  
Это вероятность того что мы получим верный $y_{train}$ из возможных $y_1, y_2 ... y_M$ для каждого объекта $u_{train}$  

Вероятность получить $y_i$ для каждого u = x1, x2... xN задается через модель и ее веса **w**  

Для оптимальных весов необходимо Likelihood -> MAX  
Это эквивалентно -ln(Likelihood) -> MIN  
Или Lossfunc = Loss $= -\sum \ln(p(y = y_{train} | u_{train})) -> min$ 

Для поиска минимума функции используется градиентный спуск

___
<u>Регуляризация</u>
Для линейного классификатора можно написать LossFunc = L(w, x) -> grad(LossFunc, w)  
Однако минимум Lossfunc может быть найден при разных значениях w (нет единственного решения).  
Поэтому чтобы это исправить и норм оптимизировать - делают **регуляризацию**

Lossfunc $= -\sum \ln(p) + \lambda \cdot R(w)$  

Варианты R(w)  
L1: R = |w1| + |w2| + ...  
L2: $R^2 = w_1^2 + w_2^2 + ...$    

L2 хорошо дифференциируется и помимо добавления однозначности - наказывает модель за очень большие $w_i$  
Оказывается, плавное распределение значений весов $w_1, w_2 ...$ положительно сказывается на том, чтобы  
модель не переобучалась (то есть не искала слишком частных решений которые фитят Train).  

Итак, с помощью регуляризации получаем единственную точку для оптимизации, страхуемся от переобучения  
lambda - фактор, задающий насколько сильно наказываем модель за избыточные веса. Подбирается как гиперпараметр.

#### Нейронные сети

В линейном классификаторе мы по сути делаем следующие преобразования:  
xv = (x1, ... xN)  ; $xv * w_{NM} = y_M$  

Для нейросетей используем следующие цепочки преобразований:   
tmp = xv * w_N_K1 # на выходе получим столбец величины K1  
tmp = L(tmp) # применяем некоторое нелинейное преобразование (функция активации)  
tmp = tmp * w_K1_K2 # очередное "взвешивание" матрицей - на выходе столбец K2  
tmp = L(tmp)  
...  
tmp = tmp * w_Kx_M = y_M # здесь получаем столбец вероятностей принадлежности к классам  

каждое умножение на матрицу w_x_y -> это новый **слой** обучаемой нейросети  
Функция активации L(x) нелинейная, чтобы получаемая модель Q была более гибкой и настраиваемой.  
Один из эффективных вариантов для обучения: L = Relu(x) = x if x > 0 else 0.  


<u>Аналогия с нейронами</u>  
Нейрон - это элемент принимающий на вход N сигналов типа x1, ... xN и выдающий M сигналов,  
Линейный классификатор - это простейший нейрон  
Каждый слой нейросети - матрица типа $W_{nm}$ <u>строка</u> которой является нейроном  
Строка принимает на вход вектор объекта x1,...xN и возвращает y1,...yM

___
Мы получаем модель с рядом слоев, каждый из которых содержит K_x * K_y весов в общем случае.  
Так как в каждом из слоев - линейное умножение, то можно аналитически записать LikeliHood модели.  
Далее, также как и в другой модели - считаем $\nabla Loss$, смещаем веса в направлении dw = $-\nabla Loss$ и двигаемся  
иттеративно пока не найдем минимум функции. Данные веса w_final - веса обученной нейросети

При этом расчет градиентов на каждом шаге сводится к матричным операциям со слоями и хорошо параллелится  
в плане вычислений. Поэтому обучение сетей производится на видеокартах

#### Сверточные нейронные сети

Проблема классических сетей в огромном кол-ве обучаемых параметров  
Каждый слой - это по сути матрица которая взвешивает каждый элемент  
У одного объекта обучающей выборки u может быть широкое признаковое пространство N.   
Например, если это картинка - то в ней может быть N>100k (rgb * число пикселей).  

Принцип сверточных сетей - когда в каждом слое взвешивание (свертка) с весами идет по принципу  
один нейрон слоя = некоторая область предудущего слоя (или объекта) - но не весь.  

<img src="images/cnn.png" width="500" align="left">  

Промежуточные слои сверточной сети - это новые признаки вместо исходных признаков объектов.  
Их можно рассматривать как более высокие уровни абстрации, генерируемые моделью в процессе обучения  
так, чтобы уже на них осуществлять более эффективную классификацию.  
"Подсматривая" в эти признаки, можно оценивать какие детали наиболее важны для модели

Модель состоит из слоев типа convolution (свертка с обучаемым ядром), pooling (изменение размерности слоя),
fully connected (слой, восстанавливающий размерность по числу классов)

#### Текстовые модели

<u> Общий пайплайн </u>
1. Input (word vector)  
2. Морфология (преобразования слов, падежи, окончания итд)
3. Синтаксис (сборка слов в предложения, учет правил)
4. Семантика (оценка "настроения" предложения, посыл итд)
5. Контекст (перенос смыслов между несколькими предложениями)

Имеем некоторый язык со всеми его словами (N ~ несколько млн)   
Каждое слово здесь w = (0, 0, ... 1, 0, ... 0) - огромный вектор  

Вектор слова w -> нейросеть -> не дискретный вектор wn = (0.1, -2, 3.2, ...)  
Вектора wn - это абстрактное представление модели, в которое однозначно переводится вектор w  
Вектор wn может быть меньшего размера, а также обладает свойствами семантической близости

___
<u> Обучение языковой модели - word2vec </u>  
База - предсказание слова по входному контексту.  
Мы даем модели на вход слова w1, w2, w3 ... wk -> она генерит вероятность встретить след. слово w_k+1  
Можно например взять большой объем текста (все статьи Wiki) и идти по ним скользящим окном k, формируя Train  
Точнее будет также брать слова до/после пропуска: "Старик поймал __ рыбку" (окно контекста)   
В итоге для каждых двух слов мы можем оценить некоторую вероятность их совместного соседства.  
Можно обучить модель так, чтобы итоговые вектора каждого слова были в таком пространстве, что  
скалярное произведение этих векторов будет давать как раз вероятность соседства = близость слов.  
При этом для модели - положительный факт встречи слов = текста, отрицательный = рандомные группы слов.  
Неплохая статья - https://habr.com/ru/articles/446530/  
Доп инфа по работе с языковыми моделями - https://web.stanford.edu/~jurafsky/slp3/3.pdf  
Лекция по работе в w2v - https://www.youtube.com/watch?v=U0LOSHY7U5Q

In [ ]:
# Пример обучения w2v модели на искусственном датасете. Для большей точности необходимо
# импортировать датасеты с наборами слов извне
import gensim
from gensim.models import Word2Vec

# Sample text data
sentences = [
    ['this', 'is', 'the', 'first', 'sentence', 'for', 'word2vec'],
    ['this', 'is', 'the', 'second', 'sentence'],
    ['yet', 'another', 'sentence'],
    ['one', 'more', 'sentence'],
    ['and', 'the', 'final', 'sentence']
]
# Train Word2Vec model - 
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
word_vector = model.wv['sentence']
similar_words = model.wv.most_similar('sentence')
print("Similar words to 'sentence':", similar_words)

#### Дополнительные ньюансы обучения

См подробнее в лекции - https://www.youtube.com/watch?v=OJlqsLg4E9Q

<u> Проблема выбора функции активации </u>  
Если выбрать функцию в стиле сигмоиды, то можно столкнуться с vanishing gradients -- градиенты  
при решении оптимизационной задачи быстро будут зануляться и обучение будет останавливаться.  
Функция Relu(x) лишена таких проблем

<u> Проблема инициализации весов </u>  
При случайной инициализации исходных весов модели w есть риск повышения нестабильности с каждым новым слоем.  
Так как градиенты обучения будут пропорциональны весам и будут усиливать тенденцию к росту / спаду параметров.  
Вариант - нормализировать веса после каждого слоя - приводить к распределению с тренируемыми параметрами.  
В итоге модель сама подстроится - какую лучше пост-нормализацию после каждого слоя выбирать.  
Это откроет путь к поиску существенно более стабильных конфигураций на базе большого кол-ва слоев.  
Техника называется Batch normalization  

То есть мы треним модель сразу на группе семплов - получаем от них статистику (среднее, дисперсия)  
Используем их для пост-нормализации с подбираемыми параметрами avg, std  
А при использовании обученной модели (фаза predict) -- подаем на вход один семпл  
и используем полученные за тренировку значения статистики.  

<u> Проблема переобучения </u>  
Модель не должна наделять отдельные веса из группы весов исключительными свойствами -  
это ведет к переобучению. Пример: если модель опознает по фото кошек, что это животное с хвостом  
и фичу типа "хвост" определяет как ключевую - то легко может переобучиться. Нужно сказать ей что то  
вроде "смотри на хвост, но и на все другие признаки в совокупности". То есть штрафовать за чрезмерное  
внимание только к хвосту.  
В линейном классификаторе (см выше) решали это регуляризацией - штрафом за величину отдельных весов.  
В нейронной сети работает концепция <u> dropout </u> -- каждое обучение случайно дропаем часть (x%)  
нейронов из каждого слоя. Это не дает завязаться на одном нейроне и перераспределяет веса на всю группу

<img src="images/dropout.png" width="400" align="left">  

<u> Проблема градиентного спуска </u>  
Классический расчет градиента в некоторой точке M-мерной функции потерь - вычислительно долгая задача.  
Порой используют стохастический спуск - случайный шаг в сторону -grad по любому из измерений. Больше не  
совсем точных шагов, но зато быстрая сходимость глобально.  
Также ускоряют алгоритмы подстраивая шаг оптимизации под крутизну спуска итд.  

Когда M значительно - <u>редко</u> можно встретить локальные минимумы, но часто - седловые точки. Когда все  
производные равны нулю, но вторые производные разных знаков. В этих точках спуск сильно буксует, прежде чем  
найти в каком направлении спуститься с седла. Есть улучшения методов, которые ускоряют этот процесс.  

Для реализации градиентного спуска в нейросетях используется метод back propagation.  
Сначала задаем вектор весов w, далее рассчитываем на тренировочных данных значения на всех узлах сети.  
Затем идем в обратном направлении, корректируя веса в узлах так, чтобы минимизировать отклонение от ожидаемого сигнала.   

<u> Learning rate </u>  
Эпоха (epoch) - иттерация в течение которой все данные Train пропущены через модель, определены градиенты весов w  
Далее мы делаем изменение весов W_upd = W - learning_rate * grad(W). И идет следующая эпоха итд.  
Величина learning_rate влияет на скорость обучения - не должна быть очень низкой или высокой.  
Хорошая практика, когда сначала берем побольше, а каждые X эпох learning_rate уменьшается.

<u> Оптимизация гиперпараметров </u>  
Может быть много параметров (droout_percent, learning_rate, ...) которые надо подбирать для задачи.  
Обычно выбирается 1 fold (train_date / validation_data) и на нем после train-обучения проверяется модель.  
Можно делать grid-перебор гиперпараметров в поисках лучшего, либо random - чтобы в выборку попадало  
больше семплов потенциально важных параметров.  

Сначала перебор идет на макро масштабе, потом масштаб уточняется (по мере уточнения поиска).  

#### Обучение с подкреплением

<u> Общая идея </u>  
Обучаемая система - некоторый агент взаимодействующий с внешней средой, который совершает последовательно   действия (a=action) и на каждое действие получает от среды подкрепление (r=reward) - полож. или отрицательное.  
Сессия взаимодействия агента со средой длится N иттераций (s = session). Агент учится так, чтобы  
среднее подкрепление в течение сессии было максимальным.  

Пример: обучение игре в аэрохоккей (коричневый скрин справа).  
Передаем на вход агенту разницу previous/next изображений, ограничиваем взаимодействие N=50 шагами (сессия)  
В конце сессии, если агент выиграл (шарик не упал) - дает reward>0 всем шагам. И наборот при поражении.  
Прогоняем обучение много раз, чтобы агент делал действия (вверх/вниз) так, чтобы максимизировать reward за N=50  
PS. reward за шаги может быть распределен неравномерно (затухать по мере увеличения N - для стабильности)  

<img src="images/reinforcement_learning.png" width="500" align="left">  

#### Рекуррентные сети (RNN)

Классические нейронные сети имеют только один вход (один объект u) -> и несколько выходов (Y)  
Хочется организовать возможность передавать на вход сети последовательность объектов u1, u2, ...   
и получать ответы исходя из суммарного контекста.  
К примеру, в задаче машинного перевода важно учитывать входную последовательность слов и их  
порядок для генерации выходного перевода.  

Это можно решить через RNN, когда каждый слой сети на вход получает инфу от своего элемента, а также предыдущего.  
На скрине справа сеть тренируется предсказывать следующий символ по предыдущим.  
В output она выдает следующую букву последовательности hello world

Если, к примеру, обучить модель на Шекспире - то она будет пытаться говорить следующий символ (-> слово)  
так, чтобы максимально попадать в текста Шекспира (то есть по сути впитает "дух" автора).  
Теперь новые входные текста она будет дополнять в духе Шекспира.

PS. Для обучения и расчета градиентов рекурентную сеть "разматывают" в линейную во времени  
Это приводит к очень длинным цепочкам и большим вычислениям, поэтому есть архитектуры вроде LSTM,  
которая помогает облегчить вычисления

<img src="images/rnn.png" width="350" align="left">  

### LLM

In [ ]:
# Заметки на базе лекции Глеба Кудрявцева

<u>LLM и ее память</u>  
LLM = конструкция, принимающая на вход набор токенов (промпт) и отдающая набор токенов.  
Сама LLM это нейросеть с триллионами весов, обученная генерировать приемлеый промпт на базе интернета (pre-train)  
Дообучается с подкреплением после этого для того чтобы ответы были точнее, имели нужный окрас и пр (post-train)  
К примеру: ее дообучают, чтобы она отделяла инструкции вида "RULES: ...; CONTEXT: ..." итд.  
  
У обычного процессора: внешняя память -> ЦПУ (вычисления)  
Для LLM роль памяти играет контекстное окно - может быть очень большим (сотни тыс токенов)  
  
Помимо управляющего промпта для нейросети присутствует еще системный промпт:  
дополнительные инструкции, которые позволяют нейросети адекватнее реагировать на разные запросы.  
Пример - tool-calling, когда пользователь просит нарисовать картинку - нейросеть выдает текст в  
виде json с параметрами которые уже можно использовать для прорисовки
  
<u>Токены</u>  
Модель обучается на токенах = частях слов, слогах и пр. Это позволяет более гибко чем по  
словам учитывать структуру языка и лучше улавливать его закономерности.  
База токенов может включать различные языковые алфавиты (латинский, китайский, кирилллицу итд)  
Разные нейросети по разному разбивают на токены входной текст промпта - можно смотреть здесь  
https://tiktokenizer.vercel.app
  
<u>RAG</u>  
Retrieval Augmented Generation = генерация с дополненной выборкой  
Нейросеть получает промпт обогащенный контекстом интересующей базы знаний и помогает работать с ним.  
При этом используются ключевые инструкции вроде "отвечай только согласно данному контексту"  
Пример: ответы на вопросы касательно имеющейся в компании базы знаний  
  
Объем базы знаний может быть очень большим и не поместится в контекстное окно.  
Для этого база знаний разбивается на небольшие куски (~1000 символов), векторизуется (эмбединги)  
и далее в зависимости от запроса пользователя в LLM подаются семантически близкие запросу куски базы.  
  
---
Далее используем инфу от Карпатого  
https://www.youtube.com/watch?v=7xTGNNLPyMI  
<u>Pre-train process</u>  
Подготавливается тренировочный датасет - это слепок всего Интернета в текстовом виде.  
Его можно взять у специализированных парсинговых компаний вроде huggingface -  
интернет страницы парсятся, чистятся от мета информации (вроде html-тегов), контекст фильтруется  
эвристиками, фильтруются нежелательные сайты/ресурсы. Далее мы получаем огромный монолитный кусок текста в ~40ТБ.  
Текст токенизируется (см выше) - раскидывается в базис, в одной из таких систем предусмотрено 10 тыс токенов,  
т е некоторый слог/сочетание = уникальный токен (id). Токенизация также чувствительна к UPPER/LOWER CASE.  
  
Далее скользящим образом идет движение по тексту, выбирается окно случайного размера (0-10 тыс токенов,  
по разному) - это контекст. На базе него нейросеть пытается предсказать следующий токен - выдает вероятности.  
Далее на базе полученного фидбека через back propagation осуществляется подгонка весов модели так, чтобы  
увеличить score угадываний по всему датасету (пример метрики качества = perplexity)  
  
Архитектура сети задается слоями и нейронами (матрицами преобразования чисел) - об этом см раздел выше.  
Сеть тренируется эпохами (epoche) - иттерация первого прохода (forward propagation) когда для входного контекста  
на имеющихся весах вычисляется результат и обратного (back propagation) когда по вычисляемой функции потерь  
мы проходимся назад и вычисляем градиенты для весов (куда их сместить чтобы результат стал точнее)  
  
Спустя N эпох функция потерь значительно снижается - веса модели подобраны под то, чтобы лучше чувствовать  
связность текста. Теперь, если ей вбросить некоторый контекст - она будет авто-дополнять его в стиле  
pre-train выборки (к примеру, если учить ее на Шекспире, то ответы будут в его духе)  
  
Однако, на этом этапе речь НЕ идет об ассистенте - это по сути продвинутый auto-completer контекста  
При этом, автодополняющая нейросеть выучила особенности Интернета (если train set 44ТБ) - и сжала их  
до 10-15 триллионов параметров-весов, что то вроде ZIP архивации  
(!) возвращает к рассуждениям о том что "интеллект - это форма архивации знаний"

### Языковые модели (todo jup notebook)

<u>Токенизация текстов</u>  
Имеем на входе текст - набор слов, символов. Хотим сопоставить ему вектор в некотором базисе.  
Один из вариантов - частотная кодировка (используется в BoW) - (1, 2, ... 0) по словарю всех слов.  
Можно использовать n-gram-ы - последовательные наборы из n слов текста - словарь сильно больше, зато  
взаимосвязи потом считать гибче. Т. е берем весь корпус текстов - и например для биграмм (n=2)  
вводим вектор всех пар слов которые последовательно встречаются в тексте.  
  
Вариант разбиения на токены - когда каждый токен = часть слова (слог или даже буква), вводится  
ориентируясь на специфику корпуса текста чтоб эффективно его закодировать.  
в итоге текст вида ("word1 word2 ...") преобразуется в (id1 id2 id3 ...) набор токенов

<u>Построение эмбеддингов</u>  
Задача: имеем похожие по смыслу слова/тексты - хотим преобразовать их в вектора фикс длины, такие что  
чем более похожи по смыслу тексты - тем ближе cos мера данных текстов.  
  
**BOW** = простой бейзлайн эмбедингов без нейросети  
BoW = bag of words. Есть вектор длиной = все используемые слова языка (неск десятков тыс например).  
Тогда некоторое слово = (0, 0, ... 1, 0, ...) sparse вектор.  
(!) BoW это частный случа n-gram, где n = 1  
Текст можно представить как = (1, 2, 0, ... 5) -> нормировать и получить частотность встречания слов.  
Не учитывается порядок слов -> близкие тексты = тексты с близкими наборами слов.  
Близость векторов можно оценивать как cos(x,y)  

**TF-IDF** = Term Frequency – Inverse Document Frequency = взвешенный вариант BOW.  
Для каждого слова x в документе d считаем координату:   
x = TF * IDF; TF = % слова x в документе d, IDF = инверсная доля документов (всех) в которых встречается слово x.   
В TF-IDF каждая координата взвешивается с учётом важности слова в документе и редкости слова в корпусе, чтобы    типичные «шумные» слова имели меньшее влияние  
К примеру, в документе много слов "и", "на" - но их и везде много - поэтому их вес понижается.  
А вот если слова специфичные для документа и используются часто - их вес в векторе растет.  
В итоге, x получает модифицированную частоту с учетом важности (в bow только частоту)  
  
**word2vec** = эмбединги через простую нейросеть  
Идея: создаем нейросеть с одним скрытым слоем = матрицей весов W.  
Пусть в обучающем корпусе N слов, учим нейросеть предсказывать вероятность того что для слова X  
слова y1, y2, ... yN-1 окажутся в контекстном окне (например рядом с ним в тексте +- 3 слова)  
Схема нейросети: входной вектор слова X (0, ... 1, ... 0) -> X * W -> emb_vec -> softmax(emb_vec)  
emb_vec - вектор фикс длины который получается при умножении на матрицу W - эмбединг слова.  
softmax переводит emb_vec в вектор вероятностей N-1: шанс что разные слова будут в контексте.  
Учим сеть, оптимизируем W. После этого можно для любого слова строить эмбединг word_vec * W   
  
Эмбединг текста здесь = агрегация эмбединга слов, то есть он контекст-независим, тоже слабый.  
  
**DSSM** = трансформеры для контекстуально зависимых эмбедингов  
Создаем размеченный датасет вида: текст X похож на текст Y итд.  
Это может быть например "запрос в поисковике" - "текст документа по которому кликнули" итд. 
Можно заранее знать какие тексты принадлежат одному сегменту и сгруппировать их - в общем, задаем похожесть.  
  
Далее, нейросеть в несколько слоев обучаем давать на выходе X, Y -> transformer -> X_emb, Y_emb такие, чтобы  
cos(X_emb, Y_emb) -> 0 для похожих документов. X, Y на входе подаем также в виде X_vec, Y_vec используя  
например n-gram или Bow.  
  
Возможны разные варианты DSSM:  
1) симметричный эмбеддинг = одна и та же сеть для X -> X_emb, Y -> Y_emb = подходит для векторизации одного  
пространства текстов чтобы потом искать семантически похожие  
2) ассиметричный = разные сети для X, Y если природа их пространств разная (например, запрос - текст)

### Пример обучения маленькой нейросети

In [1]:
import numpy as np

# --- Данные ---
tokens = ['a','b','c','d','e','f','g','h','i','j']
token_to_idx = {t:i for i,t in enumerate(tokens)}
idx_to_token = {i:t for i,t in enumerate(tokens)}

# Пример пары контекст->следующий токен
data = [
    ('a','b'),
    ('b','c'),
    ('c','d'),
    ('d','e'),
    ('e','f'),
]

def one_hot(idx, size):
    vec = np.zeros(size)
    vec[idx] = 1
    return vec

# --- Параметры сети ---
input_size = len(tokens)
hidden_size = 5
output_size = len(tokens)
lr = 0.1  # learning rate

# Случайная инициализация весов
W1 = np.random.randn(hidden_size, input_size) * 0.1
b1 = np.zeros(hidden_size)
W2 = np.random.randn(output_size, hidden_size) * 0.1
b2 = np.zeros(output_size)

# --- Функции ---
def softmax(x):
    ex = np.exp(x - np.max(x))
    return ex / ex.sum()

def forward(x):
    z1 = W1 @ x + b1
    h = np.tanh(z1)
    z2 = W2 @ h + b2
    y = softmax(z2)
    return y, h

def cross_entropy(y_pred, y_true):
    return -np.log(y_pred[y_true] + 1e-8)

# --- Обучение ---
epochs = 1000
for epoch in range(epochs):
    total_loss = 0
    for ctx, target in data:
        x = one_hot(token_to_idx[ctx], input_size)
        y_true = token_to_idx[target]

        # Forward
        y_pred, h = forward(x)
        loss = cross_entropy(y_pred, y_true)
        total_loss += loss

        # Backprop
        dy = y_pred.copy()
        dy[y_true] -= 1  # dL/dy_pred

        dW2 = np.outer(dy, h)
        db2 = dy

        dh = W2.T @ dy
        dz1 = dh * (1 - h**2)  # tanh derivative

        dW1 = np.outer(dz1, x)
        db1 = dz1

        # Update weights
        W2 -= lr * dW2
        b2 -= lr * db2
        W1 -= lr * dW1
        b1 -= lr * db1

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {total_loss/len(data):.4f}")

# --- Тест генерации ---
ctx = 'a'
for _ in range(5):
    x = one_hot(token_to_idx[ctx], input_size)
    y_pred, _ = forward(x)
    idx = np.argmax(y_pred)
    print(idx_to_token[idx], end=' ')
    ctx = idx_to_token[idx]

Epoch 0, Loss: 2.3370
Epoch 100, Loss: 0.0505
Epoch 200, Loss: 0.0174
Epoch 300, Loss: 0.0103
Epoch 400, Loss: 0.0072
Epoch 500, Loss: 0.0056
Epoch 600, Loss: 0.0045
Epoch 700, Loss: 0.0038
Epoch 800, Loss: 0.0033
Epoch 900, Loss: 0.0029
b c d e f 